# Tugas Individu BPNN

Muhammad Farhan Haniftyaji - 2006468711

link : https://colab.research.google.com/drive/1TtMS3xAOxNfyUIDqmER1nZuBHt_Ttirh?usp=sharing

## Import Library dan Dataset

In [1]:
import numpy as np
import pandas as pd
from google.colab import drive

In [2]:
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Read dataset
data = pd.read_csv('/content/drive/My Drive/dataset/Iris.csv')

## Preprocessing

In [4]:
# Delete Id column
data.drop(['Id'], axis=1)

# Encode target kelas
species_map = {
    'Iris-setosa': 0,
    'Iris-virginica': 1,
    'Iris-versicolor': 2
}

In [5]:
data['Species'] = data['Species'].map(species_map)

X = data[['SepalLengthCm', 'SepalWidthCm', 'PetalLengthCm', 'PetalWidthCm']].values
Y = np.eye(3)[data['Species'].values]

## Training

In [6]:
# Initialize weights in the range of -0.5 to 0.5
def initialize_weights(nodes):
    num_layers = len(nodes)
    weights = []

    for i in range(1, num_layers):
        num_inputs = nodes[i-1] + 1
        num_outputs = nodes[i]
        layer_weights = np.random.uniform(low=-0.5, high=0.5, size=(num_outputs, num_inputs))
        weights.append(layer_weights)

    return weights

Fungsi ini menginisialisasi bobot neural network dengan nilai random dalam range -0,5 hingga 0,5. Untuk setiap layer, fungsi menghasilkan bobot acak menggunakan fungsi distribusi seragam NumPy, dengan mempertimbangkan jumlah input dan output, dan menambahkan bobot ini ke list.

In [7]:
# Split data into training and testing sets (70:30)
def split_data(X, y, train_percent=70):
    arr_rand = np.random.rand(X.shape[0])
    split = arr_rand < np.percentile(arr_rand, train_percent)
    X_train, y_train = X[split], y[split]
    X_test, y_test = X[~split], y[~split]
    return X_train, y_train, X_test, y_test

X_train, Y_train, X_test, Y_test = split_data(X, Y)

Fungsi ini melakukan split antara train dan testing dengan banyak data yang dipakai untuk training sebesar 70%. splitting juga mengambil data secara random dari dataset untuk evaluasi pada model

In [8]:
# Processing
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def sigmoid_derivative(x):
    return np.multiply(x, 1 - x)

Fungsi sigmoid ($ \sigma(x)$) adalah fungsi aktivasi. Sigmoid memetakan bilangan asli ke kisaran antara 0 dan 1. Secara matematis didefinisikan sebagai:

$ \sigma(x) = \frac{1}{1 + e^{-x}} $

dengan :
- $e$ adalah nilai euler
- $x$ adalah input

Turunan dari sigmoid biasa digunakan pada backpropagation untuk update  weights dari neural networks selama training.

In [9]:
# Forward propagation
def forward_propagation(inputs, weights, num_layers):
    activations = [inputs]
    current_layer_input = inputs

    for layer in range(num_layers):
        layer_output = np.dot(current_layer_input, weights[layer].T)
        activation = sigmoid(layer_output)
        activations.append(activation)
        current_layer_input = np.append(1, activation)

    return activations

Fungsi ini menghitung output setiap layer dalam neural network berdasarkan data input dan bobotnya, kemudian menerapkan fungsi aktivasi sigmoid ke setiap output  layer.

In [10]:
# Backpropagation
def backpropagation(y, activations, weights, layers, lr):
    output_final = activations[-1]
    error = np.matrix(y - output_final)

    for j in range(layers, 0, -1):
        curr_activation = activations[j]

        if j > 1:
            prev_activation = np.append(1, activations[j - 1])
        else:
            prev_activation = activations[0]

        delta = np.multiply(error, sigmoid_derivative(curr_activation))
        weights[j - 1] += lr * np.multiply(delta.T, prev_activation)

        w = np.delete(weights[j - 1], [0], axis=1)
        error = np.dot(delta, w)

    return weights

Fungsi ini mengimplementasikan backpropagation untuk memperbarui bobot di neural network. Inputnya berupa target output (y), activation list setiap layer, bobot jaringan, jumlah layer, dan learning rate (lr). Dimulai dari lapisan output dan bergerak mundur, backpropagation menghitung error di setiap layer dengan mengambil selisih antara output target dan output asli, kemudian menyesuaikan bobot menggunakan gradient descent dengan learning rate. Nilai delta, yang mewakili gradien, dihitung berdasarkan turunan fungsi sigmoid, dan error pada setiap layer back propagation untuk memperbarui bobot layer sebelumnya. Terakhir, fungsi mengembalikan bobot yang diperbarui.


In [16]:
# Train function
def train(X, Y, lr, weights):
    num_layers = len(weights)

    for i in range(len(X)):
        x, y = X[i], Y[i]
        x = np.matrix(np.append(1, x))

        activations = forward_propagation(x, weights, num_layers)
        weights = backpropagation(y, activations, weights, num_layers, lr)

    return weights

Fungsi ini dilakukan untuk training neural network menggunakan gradient descent dengan input data (X), learning rate (lr), dan bobot awal layer. Fungsi ini melakukan forward propagation yang diikuti backpropagation untuk memperbarui nilai bobot berdasarkan aktivasi dan target output

In [19]:
# Train neural network
def train_neural_network(X_train, Y_train, X_val=None, Y_val=None, epochs=10, nodes=[], lr=0.15):
    hidden_layers = len(nodes) - 1
    weights = initialize_weights(nodes)
    for epoch in range(1, epochs + 1):
        weights = train(X_train, Y_train, lr, weights)
        if epoch % 10 == 0:
            print(f"Epoch {epoch}")
            print(f"Training Accuracy: {accuracy(X_train, Y_train, weights)}")
            if X_val is not None and Y_val is not None:
                print(f"Validation Accuracy: {accuracy(X_val, Y_val, weights)} \n")

    return weights

Fungsi ini melakukan training berdasarkan epoch yang di set kemudian tiap 10 epoch akan memberikan progres akurasi pada training dan validation

In [12]:
# Predict function
def predict(input_item, weights):
    num_layers = len(weights)
    input_item = np.append(1, input_item)
    activations = forward_propagation(input_item, weights, num_layers)
    final_output = activations[-1]
    max_index = np.argmax(final_output)
    prediction = np.zeros(len(final_output))
    prediction[max_index] = 1
    return prediction

Fungsi ini melakukan prediksi menggunakan neural network model yang sudah ditrain dengan menerima input dan weight. Fungsi ini menghitung aktivasi tiap layer menggunakan forward propagation.Hasil akhir diambil dari aktivasi pada layer terakhir.

Dalam kasus klasifikasi ini, fungsi akan mengidentifikasi nilai maksimum dalam vektor output yang mewakili kelas yang diprediksi. Fungsi kemudian membuat vektor prediksi di mana elemen yang sesuai dengan indeks nilai maksimum yang di set nilai 1, sementara elemen yang lain adalah 0. fungsi kemudian mengembalikan vektor prediksi.

In [1]:
# Accuracy function
def accuracy(X, Y, weights):
    correct = 0

    for i in range(len(X)):
        x, y = X[i], list(Y[i])
        guess = predict(x, weights)

        if np.array_equal(y, guess):
            correct += 1

    return correct / len(X)

Fungsi ini mencatat jumlah prediksi yang tepat dalam variable correct. Kemudian melakukan iterasi terhadap data point lalu melakukan passing terhadap input dan weight. Langkah selanjutnya adalah melakukan pembandingan antara nilai prediksi dengan target yang sebenarnya. Fungsi kemudian mengembalikan nilai yang benar terhadap jumlah data yang ada pada dataset.

In [14]:
# Define features dan output
f = len(X[0])
o = len(Y[0])

# experiment with learning rate
learning_rates = [0.1, 0.25, 0.5]
results = []

In [20]:
# Save results in list
results = []

for lr in learning_rates:
    layers = [f, 5, 10, o]
    weights = train_neural_network(X_train, Y_train, X_test, Y_test, epochs=100, nodes=layers, lr=lr)
    training_accuracy = accuracy(X_train, Y_train, weights)
    testing_accuracy = accuracy(X_test, Y_test, weights)
    results.append([lr, training_accuracy, testing_accuracy])

Epoch 10
Training Accuracy: 0.29523809523809524
Validation Accuracy: 0.4222222222222222 

Epoch 20
Training Accuracy: 0.638095238095238
Validation Accuracy: 0.7333333333333333 

Epoch 30
Training Accuracy: 0.638095238095238
Validation Accuracy: 0.7333333333333333 

Epoch 40
Training Accuracy: 0.638095238095238
Validation Accuracy: 0.7333333333333333 

Epoch 50
Training Accuracy: 0.6476190476190476
Validation Accuracy: 0.7555555555555555 

Epoch 60
Training Accuracy: 0.6761904761904762
Validation Accuracy: 0.8 

Epoch 70
Training Accuracy: 0.780952380952381
Validation Accuracy: 0.8444444444444444 

Epoch 80
Training Accuracy: 0.8285714285714286
Validation Accuracy: 0.9111111111111111 

Epoch 90
Training Accuracy: 0.8476190476190476
Validation Accuracy: 0.9333333333333333 

Epoch 100
Training Accuracy: 0.8476190476190476
Validation Accuracy: 0.9333333333333333 

Epoch 10
Training Accuracy: 0.29523809523809524
Validation Accuracy: 0.4222222222222222 

Epoch 20
Training Accuracy: 0.2952380

## Hasil Percobaan dengan Variasi Learning Rate

In [21]:
# Save the results of training in dataframe
results_df = pd.DataFrame(results, columns=['Learning Rate', 'Training Accuracy', 'Testing Accuracy'])
print(results_df)

   Learning Rate  Training Accuracy  Testing Accuracy
0           0.10           0.847619          0.933333
1           0.25           0.647619          0.755556
2           0.50           0.676190          0.777778


Dari hasil percobaan, didapatkan bahwa dengan dataset yang ada :

- nilai learning rate yang lebih kecil memiliki tingkat akurasi yang lebih tinggi dibandingkan nilai learning rate yang lebih besar
- hasil model mengindikasi terjadinya underfitting dikarenakan nilai dari training accuracy lebih rendah dibandingkan test accuracy dengan perbedaan accuracy yang cukup besar sebanyak 9-11 %